# Total / Average Emissions Benchmark
## Comparing the results of NEMED to AEMO CDEII Reporting
Using the package <code>nemed</code> to calculate daily total NEM emissions, and average emissions (or emissions intensity) over a historical period.

### Data Preparation
#### Import Packages

In [1]:
import nemed
from nemed.downloader import download_current_aemo_cdeii_summary

# To generate plots shown
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [2]:
# Open plot in browser (optional)
import plotly.io as pio
pio.renderers.default = "browser"

#### Define Parameters

In [3]:
cache = "E:/TEMPCACHE/" # Set this to an existing empty folder or leave as ""
                        # to create a new cache folder
start_time = "2021/07/01 00:00:00" # Define the start of the historical period.
end_time = "2022/07/01 00:00:00" # Define the end of the historical period.
filter_units = None # List DUIDs if you only require data for specific units.
filter_regions = None # List regions if you only require data for such.

#### Get data using NEMED

In [4]:
result = nemed.get_total_emissions(start_time, end_time, cache, filter_regions,
                                   by="day", generation_sent_out=True)

Compiling data for table DISPATCHLOAD.
Returning DISPATCHLOAD.
Compiling data for table DISPATCH_UNIT_SCADA.
Returning DISPATCH_UNIT_SCADA.
ERROR DISCREPENCY between SCADAVALUE and INITIALMW
Retrieving static table Generators and Scheduled Loads.


##### Sent Out Energy Generation

In [5]:
energy = result['Energy']
energy

REGIONID,NSW1,QLD1,SA1,TAS1,VIC1
Time,,,,,
2021-07-01,176318.99,155785.07,32358.29,35898.60,133949.88
2021-07-02,171055.62,146443.59,43762.86,28772.29,141545.39
2021-07-03,161619.58,144524.38,34189.57,27590.01,146712.20
2021-07-04,165133.89,141288.25,27904.23,28672.37,147838.99
2021-07-05,181649.29,158107.91,31777.08,41689.29,145151.71
...,...,...,...,...,...
2022-06-26,158274.46,146097.61,31257.12,34565.03,128936.49
2022-06-27,187507.48,150211.80,35670.67,39506.88,140322.68
2022-06-28,182801.87,154560.75,38172.27,39405.04,148480.12


##### Total Emissions

In [6]:
emissions_tot = result['Total_Emissions']
emissions_tot

REGIONID,NSW1,QLD1,SA1,TAS1,VIC1
Time,,,,,
2021-07-01,140718.74,118023.68,13290.66,177.28,115208.08
2021-07-02,138096.54,114928.14,7912.57,0.00,114089.28
2021-07-03,127459.32,115583.09,7469.09,0.00,125558.11
2021-07-04,124945.04,105072.08,7691.93,0.00,125344.88
2021-07-05,139919.69,115168.02,17407.86,0.00,130006.15
...,...,...,...,...,...
2022-06-26,112119.45,108708.06,9434.98,0.00,105721.45
2022-06-27,134631.24,111785.36,14469.65,221.04,115055.60
2022-06-28,130416.94,113587.46,12581.85,203.85,108928.97


##### Emissions Intensity (Average Emissions)

In [7]:
emissions_avg = result['Intensity_Index']
emissions_avg

REGIONID,NSW1,QLD1,SA1,TAS1,VIC1
Time,,,,,
2021-07-01,0.80,0.76,0.42,0.00,0.86
2021-07-02,0.81,0.79,0.17,0.00,0.81
2021-07-03,0.79,0.80,0.21,0.00,0.86
2021-07-04,0.76,0.74,0.26,0.00,0.85
2021-07-05,0.78,0.73,0.53,0.00,0.91
...,...,...,...,...,...
2022-06-26,0.71,0.74,0.29,0.00,0.83
2022-06-27,0.72,0.74,0.40,0.01,0.82
2022-06-28,0.71,0.73,0.33,0.00,0.75


##### Total Metrics of NEM

In [8]:
energy['NEM'] = energy[['NSW1','QLD1','VIC1','SA1','TAS1']].sum(axis=1)
emissions_tot['NEM'] = emissions_tot[['NSW1','QLD1','VIC1','SA1','TAS1']].sum(axis=1)
emissions_avg['NEM'] = emissions_tot['NEM'] / energy['NEM']

#### Retrieve Published AEMO CDEII data for comparison

In [9]:
aemo = download_current_aemo_cdeii_summary(start_time, end_time) # CONFIGURED FOR 2021-2022 DATA
aemo

,SETTLEMENTDATE,REGIONID,TOTAL_SENT_OUT_ENERGY,TOTAL_EMISSIONS,CO2E_INTENSITY_INDEX
0,2021-07-01,NEM,556638.63310,396835.416000,0.7129
1,2021-07-01,NSW1,186362.32480,147943.037000,0.7938
2,2021-07-01,QLD1,164005.54880,120066.631200,0.7321
3,2021-07-01,SA1,32554.46288,13284.660080,0.4081
4,2021-07-01,TAS1,36305.14318,171.734831,0.0047
...,...,...,...,...,...
2185,2022-06-30,NSW1,198876.16370,144547.716700,0.7268
2186,2022-06-30,QLD1,160751.68190,120091.864000,0.7471
2187,2022-06-30,SA1,42996.19833,11306.722490,0.2630
2188,2022-06-30,TAS1,38695.30954,0.000000,0.0000


#### Compute Error between Datasets

In [10]:
def calculate_total_emissions_error(region):
    sel_aemo = aemo[aemo['REGIONID']==region].reset_index(drop=True)
    error_calc = [emissions_tot[region].values[i] - sel_aemo['TOTAL_EMISSIONS'][i] for i in range(min(len(emissions_tot[region]), len(sel_aemo['TOTAL_EMISSIONS']))-1)]
    error_pct = [(error_calc[i] / sel_aemo['TOTAL_EMISSIONS'][i]) * 100 for i in range(len(error_calc)) if sel_aemo['TOTAL_EMISSIONS'][i]>10]
    return error_pct

def calculate_energy_error(region):
    sel_aemo = aemo[aemo['REGIONID']==region].reset_index(drop=True)
    error_calc_en = [energy[region].values[i] - sel_aemo['TOTAL_SENT_OUT_ENERGY'][i] for i in range(min(len(energy[region]), len(sel_aemo['TOTAL_SENT_OUT_ENERGY']))-1)]
    error_pct_en = [(error_calc_en[i] / sel_aemo['TOTAL_SENT_OUT_ENERGY'][i]) * 100 for i in range(len(error_calc_en))]
    return error_pct_en

def calculate_intensity_error(region):
    sel_aemo = aemo[aemo['REGIONID']==region].reset_index(drop=True)
    error_calc = [emissions_avg[region].values[i] - sel_aemo['CO2E_INTENSITY_INDEX'][i] for i in range(min(len(emissions_avg[region]), len(sel_aemo['CO2E_INTENSITY_INDEX']))-1)]
    error_pct = [(error_calc[i] / sel_aemo['CO2E_INTENSITY_INDEX'][i]) * 100 for i in range(len(error_calc))]
    return error_pct

#### Setup Plotting
##### Energy Plot Format

In [11]:
def plot_energy(region, color_idx):
    sel_aemo = aemo[aemo['REGIONID']==region].reset_index(drop=True)

    error_pct_en = calculate_energy_error(region)

    fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.05,
        specs=[[{"rowspan": 3}],[{}],[{}],[{"rowspan":1}]])
    colors = px.colors.qualitative.Dark2

    fig.update_layout(template="plotly_white", title=f"{region} Sent Out Generation<br><sub>NEM Emissions Data Tool (NEMED)</sub>",\
        font={'family':'Raleway Semibold', 'size': 12}, legend={'orientation':'h', 'xanchor': 'center', 'x': 0.5, 'y':-0.2})
    fig.update_yaxes(title_text="Total Energy (MWh)", row=1, col=1)
    fig.update_yaxes(title_text="Error wrt.<br>AEMO (%)", row=4, col=1)
    fig.update_xaxes(title_text="Date (Day)", row=4, col=1)

    fig.add_trace(go.Scatter(x=sel_aemo['SETTLEMENTDATE'],y=sel_aemo['TOTAL_SENT_OUT_ENERGY'],\
        name="AEMO CDEII Reported",mode="lines+markers",line_color=colors[7]),row=1,col=1)

    fig.add_trace(go.Scatter(x=energy.index,y=energy[region].values,\
        name="nswED Calculated",mode="lines+markers",line_color=colors[color_idx]),row=1,col=1)

    fig.add_trace(go.Scatter(x=energy.index,y=error_pct_en,\
        name="Calculation Error", mode="lines",line_color=colors[color_idx],line_dash='dot'),row=4,col=1)

    fig.show()

##### Total Emissions Plot Format

In [12]:
def plot_emissions(region, color_idx):
    sel_aemo = aemo[aemo['REGIONID']==region].reset_index(drop=True)

    error_pct = calculate_total_emissions_error(region)
        
    fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.1,
        specs=[[{"rowspan": 3}],[{}],[{}],[{"rowspan":1}]],)
    colors = px.colors.qualitative.Dark2

    fig.update_layout(template="plotly_white", title=f"{region} Total Emissions<br><sub>nsw Emissions Data Tool (NEMED)</sub>",\
        font={'family':'Raleway Semibold', 'size': 12}, legend={'orientation':'h', 'xanchor': 'center', 'x': 0.5, 'y':-0.2})
    fig.update_yaxes(title_text="Total Emissions (tCO2-e)", row=1, col=1)
    fig.update_yaxes(title_text="Error wrt.<br>AEMO (%)", row=4, col=1)
    fig.update_xaxes(title_text="Date (Day)", row=4, col=1)

    fig.add_trace(go.Scatter(x=sel_aemo['SETTLEMENTDATE'],y=sel_aemo['TOTAL_EMISSIONS'],\
        name="AEMO CDEII Report",mode="lines+markers",line_color=colors[7]),row=1,col=1)

    fig.add_trace(go.Scatter(x=emissions_tot.index,y=emissions_tot[region].values,\
        name="nswED Calculated",mode="lines+markers",line_color=colors[color_idx]),row=1,col=1)

    fig.add_trace(go.Scatter(x=emissions_tot.index[:-1],y=error_pct,\
        name="Calculation Error", mode="lines",line_color=colors[color_idx],line_dash='dot'),row=4,col=1)

    fig.show()

##### Emissions Intensity (Average Emissions) Plot Format

In [13]:
def plot_intensity(region, color_idx):
    sel_aemo = aemo[aemo['REGIONID']==region].reset_index(drop=True)

    error_pct = calculate_intensity_error(region)
        
    fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.1,
        specs=[[{"rowspan": 3}],[{}],[{}],[{"rowspan":1}]],)
    colors = px.colors.qualitative.Dark2

    fig.update_layout(template="plotly_white", title=f"{region} Average Emissions (Emissions Intensity)<br><sub>NEM Emissions Data Tool (NEMED)</sub>",\
        font={'family':'Raleway Semibold', 'size': 12}, legend={'orientation':'h', 'xanchor': 'center', 'x': 0.5, 'y':-0.2})
    fig.update_yaxes(title_text="Total Emissions (tCO2-e/MWh)", row=1, col=1)
    fig.update_yaxes(title_text="Error wrt.<br>AEMO (%)", row=4, col=1)
    fig.update_xaxes(title_text="Date (Day)", row=4, col=1)

    fig.add_trace(go.Scatter(x=sel_aemo['SETTLEMENTDATE'],y=sel_aemo['CO2E_INTENSITY_INDEX'],\
        name="AEMO CDEII Report",mode="lines+markers",line_color=colors[7]),row=1,col=1)

    fig.add_trace(go.Scatter(x=emissions_avg.index,y=emissions_avg[region].values,\
        name="NEMED Calculated",mode="lines+markers",line_color=colors[color_idx]),row=1,col=1)

    fig.add_trace(go.Scatter(x=emissions_avg.index[:-1],y=error_pct,\
        name="Calculation Error", mode="lines", line_color=colors[color_idx],line_dash='dot'),row=4,col=1)

    fig.show()

### Results - Comparison to AEMO

#### NEM

In [14]:
region = 'NEM'
plot_energy(region, color_idx=0)
plot_emissions(region, color_idx=0)
plot_intensity(region, color_idx=0)

```{include} charts_ex1/energy_nem.html
:class: full-width
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/energy_nem.html)

```{include} charts_ex1/emissions_nem.html
:class: full-width
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/emissions_nem.html)

```{include} charts_ex1/intensity_nem.html
:class: full-width
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/intensity_nem.html)

#### NSW

In [15]:
region = 'NSW1'
plot_energy(region, color_idx=1)
plot_emissions(region, color_idx=1)
plot_intensity(region, color_idx=1)

```{include} charts_ex1/energy_nsw.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/energy_nsw.html)

```{include} charts_ex1/emissions_nsw.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/emissions_nsw.html)

```{include} charts_ex1/intensity_nsw.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/intensity_nsw.html)

#### VIC

In [16]:
region = 'VIC1'
plot_energy(region, color_idx=3)
plot_emissions(region, color_idx=3)
plot_intensity(region, color_idx=3)

```{include} charts_ex1/energy_vic.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/energy_vic.html)

```{include} charts_ex1/emissions_vic.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/emissions_vic.html)

```{include} charts_ex1/intensity_vic.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/intensity_vic.html)

#### QLD

In [17]:
region = 'QLD1'
plot_energy(region, color_idx=2)
plot_emissions(region, color_idx=2)
plot_intensity(region, color_idx=2)

```{include} charts_ex1/energy_qld.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/energy_qld.html)

```{include} charts_ex1/emissions_qld.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/emissions_qld.html)

```{include} charts_ex1/intensity_qld.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/intensity_qld.html)

#### SA

In [18]:
region = 'SA1'
plot_energy(region, color_idx=4)
plot_emissions(region, color_idx=4)
plot_intensity(region, color_idx=4)

```{include} charts_ex1/energy_sa.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/energy_sa.html)

```{include} charts_ex1/emissions_sa.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/emissions_sa.html)

```{include} charts_ex1/intensity_sa.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/intensity_sa.html)

#### TAS

In [19]:
region = 'TAS1'
plot_energy(region, color_idx=5)
plot_emissions(region, color_idx=5)
plot_intensity(region, color_idx=5)

C:\Users\derlu\AppData\Local\Temp\ipykernel_26716\3118524504.py:16: RuntimeWarning:

invalid value encountered in double_scalars



```{include} charts_ex1/energy_tas.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/energy_tas.html)

```{include} charts_ex1/emissions_tas.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/emissions_tas.html)

```{include} charts_ex1/intensity_tas.html
```
[<div style="text-align: right"> <button type="button">Open Plot as Fullscreen</button></div>](charts_ex1/intensity_tas.html)